In [37]:
!pip install selenium

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import time

In [38]:
# get a reference to the download menu. This will run before the page has 
# finished loading, so we stick it in a while loop and just keep looping
# until we're successful.
def find_by_id(element_id):
    while True:
        try:
            target = driver.find_element_by_id(element_id)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target

def find_by_class(element_class):
    while True:
        try:
            target = driver.find_element_by_class_name(element_class)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target



In [39]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') -- using headless mode currently leads to NoSuchElementException 
driver = webdriver.Chrome('./chromedriver', options=options)
driver.get("https://myanimelist.net/anime/38408/Boku_no_Hero_Academia_4th_Season")

In [40]:
driver.get("https://myanimelist.net/anime/36248/Dont_Cry")

### MAIN PAGE ###


#0. Title
wrapper_region = find_by_class("wrapper")
title_region = wrapper_region.find_element_by_tag_name("h1")
Title = title_region.text

# 1. Information Left Side
wrapper_region = find_by_class("wrapper")
content_region = wrapper_region.find_element_by_id("content")
bord_region = content_region.find_element_by_class_name("borderClass")
sty = bord_region.find_element_by_tag_name('div')
rows = sty.find_elements_by_tag_name('div')

'''
# Premiered and Broadcast don't show up in all, so chop if it has:
flag = 0

if (rows[13].text[:9] == 'Broadcast'):
    del rows[13]
if (rows[12].text[:9] == 'Premiered'):
    del rows[12]
'''


def raw2int(index, cutoff):
    if "A" in (rows[-(index)].text[cutoff:].replace(',', '')):
        return None
    return int(rows[-(index)].text[cutoff:].replace(',', ''))

def duration(st):
    minutes = 0
    if "sec" in st:
        minutes = int(st.split(" sec")[0]) / 60
        return minutes
    elif ("hr" in st) and ("min" in st):
        minutes = int(st.split(" hr")[0]) * 60
        rest = st.split(" hr")[1][2:]
        return int(rest.split(" min")[0]) + minutes
    elif "hr" in st:
        return int(st.split(" hr")[0]) * 60
    elif st == "Unknown":
        return None
    elif "min" in st:
        return int(st.split(" min")[0]) + minutes

while (rows[-1].text[:10] != 'Favorites:'):
    rows.remove(rows[-1])
    
#'''
Source = rows[-11].text[8:]
Genres = rows[-10].text.split(": ")[1:][0].split(", ")
Duration = duration(rows[-9].text[10:])

if ("A" in rows[-7].text[7:]):
    ScoredCount = None
    Score = None
else:
    ScoredCount = int(rows[-7].text[7:].split(" (")[1].split("by ")[1].split(" users")[0].replace(',', ''))
    Score = float(rows[-7].text[7:].split(" (")[0])

Ranked = raw2int(5, 9)
Popularity = raw2int(3, 13)
Members = raw2int(2, 9)
Favorites = raw2int(1, 10)

for row in rows:
    print("C " + row.text.split(":")[0])
    tag = row.text.split(":", 1)[0]
    try:
        content = row.text.split(": ", 1)[1]
    except:
        content = ""
    
    if tag == "Episodes":
        Episodes = content
        if "Unknown" not in Episodes:
            Episodes = int(Episodes)
    elif tag == "Type":
        Type = content
    elif tag == "Premiered":
        premier_split = content.split(" ")
        Premiered = content
        Premiered_Season = premier_split[0]
        Premiered_Year = premier_split[1]
    elif tag == "Aired":
        aired_split = content.split(" to ")
        Aired = content
        Aired_Start_Year = aired_split[0][-4:]
        try:
            Aired_End_Year = aired_split[1][-4:]
        except:
            Aired_End_Year = Aired_Start_Year
    elif tag == "Broadcast":
        Broadcast = content
    elif tag == "Studios":
        Studios = content
    elif tag == "English":
        English_Alt = content
    elif tag == "Japanese":
        Japanese_Alt = content
    elif tag == "Synonyms":
        Synonyms = content
#'''

#'''
for row in rows[-12:]:
    print(row.text)
print(Source, Genres, Duration, Score, Ranked, Popularity, Members, Favorites, Episodes, Type)
print(Premiered, Premiered_Season, Premiered_Year, Aired, Aired_Start_Year, Aired_End_Year)
print(Broadcast, Studios, English_Alt, Japanese_Alt)
#'''


C 
C Add to My List
C 
C 
C Add to Favorites
C 
C English
C Synonyms
C Japanese
C Type
C Episodes
C Status
C Aired
C Producers
C Licensors
C Studios
C Source
C Genres
C Duration
C Rating
C Score
C 
C Ranked
C 
C Popularity
C Members
C Favorites
Studios: None found, add some
Source: Manga
Genres: Music, Slice of Life, School, Seinen
Duration: 5 min.
Rating: G - All Ages
Score: N/A1 (scored by - users)

Ranked: #141662

Popularity: #14893
Members: 222
Favorites: 0
Manga ['Music', 'Slice of Life', 'School', 'Seinen'] 5 None 141662 14893 222 0 1 Music
Spring 2015 Spring 2015 Jun, 2017 2017 2017
Not scheduled once per week None found, add some Don't cry Don't cry


In [41]:
def duration(st):
    if "hr" in st:
        minutes = int(st.split(" hr")[0]) * 60
        rest = st.split(" hr")[1][2:]
    elif st == "Unknown":
        return None
    else:
        rest = st

    return int(rest.split(" min")[0]) + minutes

duration(rows[-9].text[10:])

UnboundLocalError: local variable 'minutes' referenced before assignment

In [ ]:
# "One" hot encoding for Genres

# TO be used at very end, after all scrapes are done
#https://stackoverflow.com/questions/62408093/one-hot-encoding-multiple-categorical-data-in-a-column

df = pd.DataFrame()
df["Genres"] = [Genres, ['Action', 'Military']]
df["Title"] = ["Wee", "blob"]

#df['Genres'].fillna('NA', inplace = True)
flat_genre = [item for sublist in df["Genres"] for item in sublist]
set_genre = set(flat_genre)
unique_genre = list(set_genre)
#unique_genre.remove('NA')
df.reindex(derf.columns.tolist() + unique_genre, axis=1, fill_value=0)

for index, row in df.iterrows():
    for val in row.Genres:
        df.loc[index, val] = 1


df.fillna(0)

In [61]:
driver.get("https://myanimelist.net/anime/42361/Ijiranaide_Nagatoro-san")

# 2A. Information Right Side - Synopsis
wrapper_region = find_by_class("wrapper")
content_region = wrapper_region.find_element_by_id("content")
bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
synopsis_region = bord_region.find_element_by_tag_name('p')
Synopsis = synopsis_region.text

In [62]:
# 2B. Information Right Side - Staff
big_region = bord_region.find_elements_by_class_name('pb24')[-1]
char_region = big_region.find_elements_by_tag_name("div")[4]
va_region = char_region.find_elements_by_class_name('borderClass')

k = 0
va_list = []


for entry in va_region:
    if (k % 3 == 2):
        va_list.append(entry.text.split("\n")[0])
    k += 1
    
va_list

['Uesaka, Sumire',
 'Yamashita, Daiki',
 'Suzuki, Aina',
 'Komatsu, Mikako',
 'Izawa, Shiori',
 '']

In [63]:
net_df = pd.DataFrame()

# 2C. Information Right Side - Prequel/Sequel
big_region = bord_region.find_elements_by_class_name('pb24')[-1]
char_region = big_region.find_element_by_class_name("anime_detail_related_anime")
related_region = char_region.find_elements_by_tag_name('tr')

Prequel = None
Sequel = None
for i in related_region:
    if ("Prequel:" in i.text):
        Prequel = i.text[9:]
        First = False
        net_df = net_df.append({"Source":Prequel, "Target":Title, "Weight":1}, ignore_index=True)
    else:
        First = True
    if ("Sequel: " in i.text):
        Sequel = i.text[8:]
        net_df = net_df.append({"Source":Title, "Target":Sequel, "Weight":1}, ignore_index=True)

print(Prequel, Sequel, First)
net_df

None None True


""


In [205]:
# 3. Stats
driver.get("https://myanimelist.net/anime/20/Naruto/stats")

wrapper_region = find_by_class("wrapper")
content_region = wrapper_region.find_element_by_id("content")
bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
stat_region = bord_region.find_elements_by_class_name('spaceit_pad')

    
def stat2int(index, cutoff):
    return int(stat_region[index].text[cutoff:].replace(',', ''))

def stat2score(index):
    return int(stat_region[index].text.replace('v', '(').split(' (')[1])

Watching = stat2int(0, 9)
Completed = stat2int(1, 11)
On_Hold = stat2int(2, 9)
Dropped = stat2int(3, 9)
Plan_to_Watch = stat2int(4, 15)
Total = stat2int(5, 7)

if (len(stat_region) == 16):
    s10 = stat2score(6)
    s9 = stat2score(7)
    s8 = stat2score(8)
    s7 = stat2score(9)
    s6 = stat2score(10)
    s5 = stat2score(11)
    s4 = stat2score(12)
    s3 = stat2score(13)
    s2 = stat2score(14)
    s1 = stat2score(15)
else:
    s10 = s9 = s8 = s7 = s6 = s5 = s4 = s3 = s2 = s1 = None

'''
for i in stat_region:
    print(i.text)
print(Watching, Completed, On_Hold, Dropped, Plan_to_Watch, Total)
print(s10, s9, s8, s7, s6, s5, s4, s3, s2, s1)
'''


Watching: 148,505
Completed: 1,570,783
On-Hold: 66,219
Dropped: 103,889
Plan to Watch: 75,943
Total: 1,965,339
 17.2% (234520 votes)
 18.6% (253455 votes)
 27.3% (372837 votes)
 22.5% (307666 votes)
 8.4% (115091 votes)
 3.6% (49417 votes)
 1.2% (16176 votes)
 0.5% (6376 votes)
 0.3% (3721 votes)
 0.4% (5635 votes)
16


In [47]:
# 4. Episodes
driver.get("https://myanimelist.net/anime/3712/Zero_no_Tsukaima__Princesses_no_Rondo/episode")
wrapper_region = find_by_class("wrapper")
content_region = wrapper_region.find_element_by_id("content")
bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
stat_region = bord_region.find_element_by_tag_name('td')
bs_region = stat_region.find_element_by_class_name("border_solid")
ep_region = bs_region.find_elements_by_tag_name("span")[-1]
Episodes = int(ep_region.text.replace(")", "/").split("/")[1])


12

In [137]:
# 5. Recommendations (net_df exclusive)
driver.get("https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/userrecs")
wrapper_region = find_by_class("wrapper")
content_region = wrapper_region.find_element_by_id("content")
bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
rec_region = bord_region.find_elements_by_css_selector("[class='borderClass']")

sources = []
targets = []
weights = []
weight_total = 0

for rec in rec_region:

    sources.append("blah")
    targets.append(rec.find_elements_by_tag_name("div")[3].text.split(" add")[0])
    
    try:
        weights.append(int(rec.find_element_by_class_name("spaceit").text.split(" more")[0].split("by ")[1]))
    except:
        weights.append(1)
    weight_total += weights[-1]
    
temp_df = pd.DataFrame(list(zip(sources, targets, weights)), columns=["Source", "Target", "Weight"])
temp_df["Weight"] = temp_df["Weight"] / weight_total

net_df = net_df.append(temp_df, ignore_index = True)

,Source,Target,Weight
0,Shingeki no Kyojin,Shingeki no Kyojin Season 2,1.000000
1,blah,Hunter x Hunter (2011),0.218527
2,blah,Shingeki no Kyojin,0.083135
3,blah,D.Gray-man,0.047506
4,blah,Ao no Exorcist,0.035629
...,...,...,...
149,blah,Trigun,0.002375
150,blah,Strait Jacket,0.002375
151,blah,Katekyo Hitman Reborn!,0.002375
152,blah,Full Metal Panic!,0.002375


Putting it together as a df: https://pythonexamples.org/pandas-dataframe-add-append-row/

In [45]:
def page_scrape(df, link, net_df):
    
    
    malrow = {}
    
    ### MAIN PAGE ###

    
    driver.get(link)  
    
    #0. Title
    wrapper_region = find_by_class("wrapper")
    title_region = wrapper_region.find_element_by_tag_name("h1")
    malrow["Title"] = title_region.text
    
    # Hyperlink
    malrow['Hyperlink'] = '=HYPERLINK("%s", "%s")' % (link.format(malrow["Title"]), malrow["Title"])

    # 1. Information Left Side
    wrapper_region = find_by_class("wrapper")
    content_region = wrapper_region.find_element_by_id("content")
    bord_region = content_region.find_element_by_class_name("borderClass")
    sty = bord_region.find_element_by_tag_name('div')
    rows = sty.find_elements_by_tag_name('div')

    def duration(st):
        minutes = 0
        if "sec" in st:
            minutes = int(st.split(" sec")[0]) / 60
            return minutes
        elif ("hr" in st) and ("min" in st):
            minutes = int(st.split(" hr")[0]) * 60
            rest = st.split(" hr")[1][2:]
            return int(rest.split(" min")[0]) + minutes
        elif "hr" in st:
            return int(st.split(" hr")[0]) * 60
        elif st == "Unknown":
            return None
        elif "min" in st:
            return int(st.split(" min")[0]) + minutes

    
    def raw2int(index, cutoff):
        if "A" in (rows[-(index)].text[cutoff:].replace(',', '')):
            return None
        return int(rows[-(index)].text[cutoff:].replace(',', ''))
    
    while (rows[-1].text[:10] != 'Favorites:'):
        rows.remove(rows[-1])

    malrow["Source"] = rows[-11].text[8:]
    malrow["Genres"] = rows[-10].text.split(": ")[1:][0].split(", ")
    malrow["Duration"] = duration(rows[-9].text[10:])
    
    if ("A" in rows[-7].text[7:]):
        malrow["ScoredCount"] = None
        malrow["Score"] = None
    else:
        malrow["ScoredCount"] = int(rows[-7].text[7:].split(" (")[1].split("by ")[1].split(" users")[0].replace(',', ''))
        malrow["Score"] = float(rows[-7].text[7:].split(" (")[0])
    
    malrow["Ranked"] = raw2int(5, 9)
    malrow["Popularity"] = raw2int(3, 13)
    malrow["Members"] = raw2int(2, 9)
    malrow["Favorites"] = raw2int(1, 10)
    
    for row in rows:
        tag = row.text.split(":", 1)[0]
        try:
            content = row.text.split(": ", 1)[1]
        except:
            content = ""

        if tag == "Episodes":
            malrow["Episodes"] = content
            if "Unknown" not in malrow["Episodes"]:
                malrow["Episodes"] = int(malrow["Episodes"])
        elif tag == "Type":
            malrow["Type"] = content
        elif tag == "Premiered":
            premier_split = content.split(" ")
            malrow["Premiered"] = content
            malrow["Premiered Season"] = premier_split[0]
            malrow["Premiered Year"] = premier_split[1]
        elif tag == "Aired":
            aired_split = content.split(" to ")
            malrow["Aired"] = content
            malrow["Start Year"] = aired_split[0][-4:]
            try:
                malrow["End Year"] = aired_split[1][-4:]
            except:
                malrow["End Year"] = malrow["Start Year"]
        elif tag == "Broadcast":
            malrow["Broadcast"] = content
        elif tag == "Studios":
            malrow["Studios"] = content
        elif tag == "English":
            malrow["English Alternative Names"] = content
        elif tag == "Japanese":
            malrow["Japanese Alternative Names"] = content
        elif tag == "Synonyms":
            malrow["Synonyms"] = content

    # 2A. Information Right Side - Synopsis
    wrapper_region = find_by_class("wrapper")
    content_region = wrapper_region.find_element_by_id("content")
    bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
    synopsis_region = bord_region.find_element_by_tag_name('p')
    malrow["Synopsis"] = synopsis_region.text
    
    '''
    # 2B. Information Right Side - Staff
    big_region = bord_region.find_elements_by_class_name('pb24')[-1]
    char_region = big_region.find_elements_by_tag_name("div")[4]
    va_region = char_region.find_elements_by_class_name('borderClass')

    k = 0
    va_list = []
    for entry in va_region:
        if (k % 3 == 2):
            va_list.append(entry.text.split("\n")[0])
        k += 1

    malrow["Voice Actors"] = va_list
    '''
    
    # 2C. Information Right Side - Prequel/Sequel
    big_region = bord_region.find_elements_by_class_name('pb24')[-1]
    malrow["First"] = True
    
    preq_include = False
    seq_include = False
    try:
        char_region = big_region.find_element_by_class_name("anime_detail_related_anime")
        related_region = char_region.find_elements_by_tag_name('tr')

        malrow["Prequel"] = None
        malrow["Sequel"] = None
        for i in related_region:
            if ("Prequel:" in i.text):
                malrow["Prequel"] = i.text[9:]
                malrow["First"] = False
                preq_include = True
            if ("Sequel: " in i.text):
                malrow["Sequel"] = i.text[8:]
                seq_include = True
    except:
        malrow["Prequel"] = None
        malrow["Sequel"] = None
    
    
    time.sleep(np.random.rand() * 3 + 2)
    
    ### STATS ###
    
    # 3. Stats
    driver.get(link + "/stats")

    wrapper_region = find_by_class("wrapper")
    content_region = wrapper_region.find_element_by_id("content")
    bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
    stat_region = bord_region.find_elements_by_class_name('spaceit_pad')


    def stat2int(index, cutoff):
        return int(stat_region[index].text[cutoff:].replace(',', ''))

    def stat2score(index):
        return int(stat_region[index].text.replace('v', '(').split(' (')[1])

    malrow["Watching"] = stat2int(0, 9)
    malrow["Completed"] = stat2int(1, 11)
    malrow["On-Hold"] = stat2int(2, 9)
    malrow["Dropped"] = stat2int(3, 9)
    malrow["Plan to Watch"] = stat2int(4, 15)
    malrow["Total"] = stat2int(5, 7)
    
    
    if (len(stat_region) == 16):
        sum = 0
        for i in range(1, 11):
            malrow[str(i)] = stat2score(16 - i)
            sum += stat2score(16 - i)
        malrow["ScoredCount"] = sum
        for i in range(1, 11):
            malrow[str(i)] = malrow[str(i)] / sum * 100
    else:
        for i in range(1, 11):
            malrow[str(i)] = None
        
    
    '''
    # 4. Episodes
    driver.get(link + "/episode")
    wrapper_region = find_by_class("wrapper")
    content_region = wrapper_region.find_element_by_id("content")
    bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel") ##
    stat_region = bord_region.find_element_by_tag_name('td')
    bs_region = stat_region.find_element_by_class_name("border_solid")
    ep_region = bs_region.find_elements_by_tag_name("span")[-1]
    malrow["Episodes"] = int(ep_region.text.replace(")", "/").split("/")[1])
    '''

    time.sleep(np.random.rand() * 3 + 2)
    
    # 5. Recommendations (net_df exclusive)
    driver.get(link + "/userrecs")
    wrapper_region = find_by_class("wrapper")
    content_region = wrapper_region.find_element_by_id("content")
    bord_region = content_region.find_element_by_class_name("js-scrollfix-bottom-rel")
    rec_region = bord_region.find_elements_by_css_selector("[class='borderClass']")

    sources = []
    targets = []
    weights = []
    weight_total_filler = []
    weight_total = 0

    for rec in rec_region:

        sources.append(malrow["Title"])
        targets.append(rec.find_elements_by_tag_name("div")[3].text.split(" add")[0])

        try:
            weights.append(int(rec.find_element_by_class_name("spaceit").text.split(" more")[0].split("by ")[1]))
        except:
            weights.append(1)
        weight_total += weights[-1]
        weight_total_filler.append(1)

    temp_df = pd.DataFrame(list(zip(sources, targets, weights, weights, weight_total_filler)), 
                           columns=["Source", "Target", "Weight", "Raw", "Weight Total"])
    temp_df["Weight"] = temp_df["Weight"] / weight_total
    temp_df["Weight Total"] = temp_df["Weight Total"] * weight_total

    if preq_include:
        net_df = net_df.append({"Source":malrow["Prequel"], "Target":malrow["Title"], "Weight":1, "Raw":-1, "Weight Total":weight_total}, ignore_index=True)
    if seq_include:
        net_df = net_df.append({"Source":malrow["Title"], "Target":malrow["Sequel"], "Weight":1, "Raw":-1, "Weight Total":weight_total}, ignore_index=True)
    
    net_df = net_df.append(temp_df, ignore_index = True)

    time.sleep(np.random.rand() * 3 + 2)
    
    # Final steps
    df = df.append(malrow, ignore_index=True)
    return df, net_df

In [46]:
net_df = pd.DataFrame()

df, net_df = page_scrape(pd.DataFrame(columns=[
    "Title", "Episodes", "Score", "Synopsis", "Source", "Genres", "Duration", 
    "Ranked", "Popularity", "Members", "Favorites",
    "Watching", "Completed", "On-Hold", "Dropped", "Plan to Watch", "Total", "1", "2", "3", "4", "5", "6", "7",
    "8", "9", "10", "ScoredCount"
]), "https://myanimelist.net/anime/42625/Heion_Sedai_no_Idaten-tachi", net_df)
df

,Title,Episodes,Score,Synopsis,Source,Genres,Duration,Ranked,Popularity,Members,...,Japanese Alternative Names,Premiered,Premiered Season,Premiered Year,Prequel,Sequel,Start Year,Studios,Synonyms,Type
0,Heion Sedai no Idaten-tachi,Unknown,None,"It has been 800 years since the battle gods ""I...",Manga,"[Action, Adventure, Demons, Fantasy, Seinen]",None,None,3883,19292,...,平穏世代の韋駄天達,Summer 2021,Summer,2021,None,None,2021,MAPPA,Idaten Deities in the Peaceful Generation,TV


In [47]:
df, net_df  = page_scrape(df, "https://myanimelist.net/anime/35286/Henkei_Shoujo", net_df)
df, net_df  = page_scrape(df, "https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood", net_df)
df, net_df  = page_scrape(df, "https://myanimelist.net/anime/16498/Shingeki_no_Kyojin", net_df)
df

,Title,Episodes,Score,Synopsis,Source,Genres,Duration,Ranked,Popularity,Members,...,Premiered,Premiered Season,Premiered Year,Prequel,Sequel,Start Year,Studios,Synonyms,Type,English Alternative Names
0,Heion Sedai no Idaten-tachi,Unknown,None,"It has been 800 years since the battle gods ""I...",Manga,"[Action, Adventure, Demons, Fantasy, Seinen]",None,None,3883,19292,...,Summer 2021,Summer,2021,None,None,2021,MAPPA,Idaten Deities in the Peaceful Generation,TV,NaN
1,Henkei Shoujo,5,5.711,Shorts about girls who transform into machines...,Original,[Sci-Fi],1,92732,3809,19748,...,NaN,NaN,NaN,None,None,2017,DLE,"Transforming Girls, Henkei Girls",ONA,Henkei Shojo
2,Fullmetal Alchemist: Brotherhood,64,9.181,"""In order for something to be obtained, someth...",Manga,"[Action, Military, Adventure, Comedy, Drama, M...",24,12,3,2393923,...,Spring 2009,Spring,2009,None,None,2009,Bones,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",TV,Fullmetal Alchemist: Brotherhood
3,Shingeki no Kyojin,25,8.501,"Centuries ago, mankind was slaughtered to near...",Manga,"[Action, Military, Mystery, Super Power, Drama...",24,1072,2,2754310,...,Spring 2013,Spring,2013,None,Shingeki no Kyojin Season 2,2013,Wit Studio,AoT,TV,Attack on Titan


In [48]:
df.columns

Index(['Title', 'Episodes', 'Score', 'Synopsis', 'Source', 'Genres',
       'Duration', 'Ranked', 'Popularity', 'Members', 'Favorites', 'Watching',
       'Completed', 'On-Hold', 'Dropped', 'Plan to Watch', 'Total', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', '10', 'ScoredCount', 'Aired',
       'Broadcast', 'End Year', 'First', 'Hyperlink',
       'Japanese Alternative Names', 'Premiered', 'Premiered Season',
       'Premiered Year', 'Prequel', 'Sequel', 'Start Year', 'Studios',
       'Synonyms', 'Type', 'English Alternative Names'],
      dtype='object')

In [142]:
net_df

,Source,Target,Weight
0,Henkei Shoujo,Inuyashiki,0.250000
1,Henkei Shoujo,Pupa,0.250000
2,Henkei Shoujo,Love Kome: We Love Rice,0.250000
3,Henkei Shoujo,Frame Arms Girl,0.250000
4,Fullmetal Alchemist: Brotherhood,Hunter x Hunter (2011),0.218527
...,...,...,...
321,Shingeki no Kyojin,Mushibugyou,0.001211
322,Shingeki no Kyojin,Another,0.001211
323,Shingeki no Kyojin,Freedom,0.001211
324,Shingeki no Kyojin,Yuri Kuma Arashi,0.001211


In [143]:
# Final conversion


#df['Genres'].fillna('NA', inplace = True)
flat_genre = [item for sublist in df["Genres"] for item in sublist]
set_genre = set(flat_genre)
unique_genre = list(set_genre)
#unique_genre.remove('NA')
df.reindex(derf.columns.tolist() + unique_genre, axis=1, fill_value=0)

for index, row in df.iterrows():
    for val in row.Genres:
        df.loc[index, val] = 1


df = df.fillna(0)
df

,Title,Episodes,Score,Synopsis,Voice Actors,Source,Genres,Duration,Ranked,Popularity,...,Romance,Ecchi,Fantasy,School,Sci-Fi,Military,Drama,Shounen,Mystery,Super Power
0,Zero no Tsukaima: Princesses no Rondo,12,7.381,Following his brave sacrifice in the war again...,"[Kugimiya, Rie, Hino, Satoshi, Inokuchi, Yuka,...",Light novel,"[Action, Adventure, Harem, Comedy, Magic, Roma...",24,20122,382,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Henkei Shoujo,5,5.711,Shorts about girls who transform into machines...,"[Enako, Michishige, Sayumi, Hanazawa, Kana, Ao...",Original,[Sci-Fi],1,92472,3808,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Fullmetal Alchemist: Brotherhood,64,9.181,"""In order for something to be obtained, someth...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ...",Manga,"[Action, Military, Adventure, Comedy, Drama, M...",24,12,3,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,Shingeki no Kyojin,25,8.501,"Centuries ago, mankind was slaughtered to near...","[Kaji, Yuki, Ishikawa, Yui, Inoue, Marina, Kam...",Manga,"[Action, Military, Mystery, Super Power, Drama...",24,1042,2,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [144]:
df.to_csv('PageScrape.csv')